# Многоклассовая классификация обращений клиентов на сайте banki.ru по брокерскому обслуживанию Сбербанка

Причины выбора алгоримта:
- производственная необходимость
- отработка навыков обработки текста
- практика использования методов ML

Порядок работы:
1. Получение реальных обращений с сайта banki.ru по всем кредитным организациям для целей обучения модели
2. Подготовка и предобработка данных
3. Токенизация и стемминг
4. Токенизация и лемматизация
5. Стемминг и лемматизация: что лучше? Выбор подхода в каждой из моделей
    - Naive Bayes
    - Linear Support Vector Machine
    - Logistic Regression
6. Получение реальных обращений клиентов, касающихся Сбербанка
7. Классификация обращений выбранным подходом
8. Выводы

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import ssl
import time as t

Загрузка жалоб клиентов по всем банкам с целью получения данных для обучения

In [2]:
url = 'https://www.banki.ru/services/responses/list/product/investments/'
r = requests.get(url)

df_allbanks = pd.DataFrame(columns=['url жалобы', 'Заголовок', 'Текст', 'Время', 'Оценка'])

soup = BeautifulSoup(r.text)

pages = 50
page_url= '?page='
for page in range(1, pages + 2):
    t.sleep(1)
    for article in soup.findAll('article'):
        try:
            href = article.find('a', {'data-test': 'responses-header'}).get('href')
            header = article.find('a', {'data-test': 'responses-header'}).text
            text = article.find('div', {'data-test': 'responses-message'}).text.strip()
            time = article.find('time', {'data-test': 'responses-datetime'}).text.strip()
            
            # optional
            try:
                rating = article.find('span', {'data-test': 'responses-rating-grade'}).text.strip()
            except:
                rating = None
            row = [url[:21]+href[1:], header, text, time, rating]
            df_allbanks.loc[len(df_allbanks)] = row
        except:
            pass
    # Взятие следующей страницы
    try:
        if page <= pages:
            r = requests.get(url + page_url + str(page))
            soup = BeautifulSoup(r.text, features='lxml')
    except:
        pass
print(len(df_allbanks))
# Промежуточное сохранение
df_allbanks.to_excel('Отзывы_все_банки.xlsx')

1275


Подготовка и предобработка данных

In [3]:
df_train=pd.read_excel('Отзывы_все_банки_разметка.xlsx')
df_train.drop(['Unnamed: 0'], axis=1, inplace=True)
#df_train=df_allbanks
df_train

,url жалобы,Заголовок,Текст,Время,Оценка,Тематика
0,https://www.banki.ru/services/responses/bank/r...,Отличный банк!\n\t\t\t\t\t\t\t\t,Я давно сотрудничаю с этим банком. Специалисты...,15.05.2021 14:50,5.0,благодарность
1,https://www.banki.ru/services/responses/bank/r...,БУДЬТЕ ОЧЕНЬ БДИТЕЛЬНЫМИ в этом банке!Не реко...,"Придя в банк,я заняла очередь.Моим вопросом мо...",15.05.2021 14:44,2.0,плохое обслуживание
2,https://www.banki.ru/services/responses/bank/r...,"Названивают без повода, не слушают вообще ниче...",Названивают мне с информацией о незнакомом мне...,15.05.2021 13:24,1.0,навязывание
3,https://www.banki.ru/services/responses/bank/r...,Благодарность\n\t\t\t\t\t\t\t\t,"Обращался в Банк Тинькофф по поводу акции ""При...",15.05.2021 10:00,5.0,благодарность
4,https://www.banki.ru/services/responses/bank/r...,Отвратительное отношение к клиентам\n\t\t\t\t\...,Обратился в банк т.к. появилось желания начать...,14.05.2021 23:23,1.0,плохое обслуживание
...,...,...,...,...,...,...
1149,https://www.banki.ru/services/responses/bank/r...,Самый удобный банк!\n\t\t\t\t\t\t\t\t,Хотелось бы выразить благодарность банку и сот...,04.03.2021 14:19,5.0,благодарность
1150,https://www.banki.ru/services/responses/bank/r...,Открытие брокерского счета длилось почти месяц...,10.02 я подала заявку на открытие брокерского ...,04.03.2021 14:10,2.0,некачественная услуга
1151,https://www.banki.ru/services/responses/bank/r...,Обманули и оформили без спроса брокерский счет...,"Здравствуйте! Полное безобразие, сегодня была,...",04.03.2021 13:56,1.0,навязывание
1152,https://www.banki.ru/services/responses/bank/r...,По глупости перевел деньги обманщикам\n\t\t\t\...,Недавно перевел небольшую сумму мошенникам с к...,04.03.2021 12:26,5.0,благодарность


In [4]:
df_train['Заг_и_текст']=df_train['Заголовок']+df_train['Текст']

In [5]:
import string
def remove_punctuation(text):
    return "".join([ch if ch not in string.punctuation else ' ' for ch in text])

def remove_numbers(text):
    return ''.join([i if not i.isdigit() else ' ' for i in text])

import re
def remove_multiple_spaces(text):
    return re.sub(r'\s+', ' ', text, flags=re.I)

In [6]:
prep_text = [remove_multiple_spaces(remove_numbers(remove_punctuation(text.lower()))) for text in df_train['Заг_и_текст']]

df_train['Пред_текст'] = prep_text
df_train.head()

,url жалобы,Заголовок,Текст,Время,Оценка,Тематика,Заг_и_текст,Пред_текст
0,https://www.banki.ru/services/responses/bank/r...,Отличный банк!\n\t\t\t\t\t\t\t\t,Я давно сотрудничаю с этим банком. Специалисты...,15.05.2021 14:50,5.0,благодарность,Отличный банк!\n\t\t\t\t\t\t\t\tЯ давно сотруд...,отличный банк я давно сотрудничаю с этим банко...
1,https://www.banki.ru/services/responses/bank/r...,БУДЬТЕ ОЧЕНЬ БДИТЕЛЬНЫМИ в этом банке!Не реко...,"Придя в банк,я заняла очередь.Моим вопросом мо...",15.05.2021 14:44,2.0,плохое обслуживание,БУДЬТЕ ОЧЕНЬ БДИТЕЛЬНЫМИ в этом банке!Не реко...,будьте очень бдительными в этом банке не реком...
2,https://www.banki.ru/services/responses/bank/r...,"Названивают без повода, не слушают вообще ниче...",Названивают мне с информацией о незнакомом мне...,15.05.2021 13:24,1.0,навязывание,"Названивают без повода, не слушают вообще ниче...",названивают без повода не слушают вообще ничег...
3,https://www.banki.ru/services/responses/bank/r...,Благодарность\n\t\t\t\t\t\t\t\t,"Обращался в Банк Тинькофф по поводу акции ""При...",15.05.2021 10:00,5.0,благодарность,Благодарность\n\t\t\t\t\t\t\t\tОбращался в Бан...,благодарность обращался в банк тинькофф по пов...
4,https://www.banki.ru/services/responses/bank/r...,Отвратительное отношение к клиентам\n\t\t\t\t\...,Обратился в банк т.к. появилось желания начать...,14.05.2021 23:23,1.0,плохое обслуживание,Отвратительное отношение к клиентам\n\t\t\t\t\...,отвратительное отношение к клиентам обратился ...


In [7]:
from nltk.stem import *
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation

mystem = Mystem() 

russian_stopwords = stopwords.words("russian")
russian_stopwords.extend(['…', '«', '»', '...', '\n', '\t', 'весь', 'отзыв'])
def lemmatize_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords and token != " "]
    text = " ".join(tokens)
    return text

Токенизация

Стемминг

In [8]:
from nltk.stem.snowball import SnowballStemmer 
stemmer = SnowballStemmer("russian")

from nltk import word_tokenize

In [9]:
%%time
stemmed_texts_list = []
for text in df_train['Пред_текст']:
    tokens = word_tokenize(text)    
    stemmed_tokens = [stemmer.stem(token) for token in tokens if token not in russian_stopwords]
    text = " ".join(stemmed_tokens)
    stemmed_texts_list.append(text)

df_train['Стем_текст'] = stemmed_texts_list

df_train.head()

Wall time: 1.7 s


,url жалобы,Заголовок,Текст,Время,Оценка,Тематика,Заг_и_текст,Пред_текст,Стем_текст
0,https://www.banki.ru/services/responses/bank/r...,Отличный банк!\n\t\t\t\t\t\t\t\t,Я давно сотрудничаю с этим банком. Специалисты...,15.05.2021 14:50,5.0,благодарность,Отличный банк!\n\t\t\t\t\t\t\t\tЯ давно сотруд...,отличный банк я давно сотрудничаю с этим банко...,отличн банк давн сотруднича эт банк специалист...
1,https://www.banki.ru/services/responses/bank/r...,БУДЬТЕ ОЧЕНЬ БДИТЕЛЬНЫМИ в этом банке!Не реко...,"Придя в банк,я заняла очередь.Моим вопросом мо...",15.05.2021 14:44,2.0,плохое обслуживание,БУДЬТЕ ОЧЕНЬ БДИТЕЛЬНЫМИ в этом банке!Не реко...,будьте очень бдительными в этом банке не реком...,будьт очен бдительн банк рекоменд прид банк за...
2,https://www.banki.ru/services/responses/bank/r...,"Названивают без повода, не слушают вообще ниче...",Названивают мне с информацией о незнакомом мне...,15.05.2021 13:24,1.0,навязывание,"Названивают без повода, не слушают вообще ниче...",названивают без повода не слушают вообще ничег...,названива повод слуша вообщ хам телефон исполь...
3,https://www.banki.ru/services/responses/bank/r...,Благодарность\n\t\t\t\t\t\t\t\t,"Обращался в Банк Тинькофф по поводу акции ""При...",15.05.2021 10:00,5.0,благодарность,Благодарность\n\t\t\t\t\t\t\t\tОбращался в Бан...,благодарность обращался в банк тинькофф по пов...,благодарн обраща банк тинькофф повод акц приве...
4,https://www.banki.ru/services/responses/bank/r...,Отвратительное отношение к клиентам\n\t\t\t\t\...,Обратился в банк т.к. появилось желания начать...,14.05.2021 23:23,1.0,плохое обслуживание,Отвратительное отношение к клиентам\n\t\t\t\t\...,отвратительное отношение к клиентам обратился ...,отвратительн отношен клиент обрат банк т появ ...


Лемматизация

In [10]:
%%time
lemm_texts_list = []
for text in df_train['Пред_текст']:
    try:
        text_lem = mystem.lemmatize(text)
        tokens = [token for token in text_lem if token != ' ' and token not in russian_stopwords]
        text = " ".join(tokens)
        lemm_texts_list.append(text)
    except Exception as e:
        print(e)
    
df_train['Лемм_текст'] = lemm_texts_list
df_train.head()

Wall time: 25min 54s


,url жалобы,Заголовок,Текст,Время,Оценка,Тематика,Заг_и_текст,Пред_текст,Стем_текст,Лемм_текст
0,https://www.banki.ru/services/responses/bank/r...,Отличный банк!\n\t\t\t\t\t\t\t\t,Я давно сотрудничаю с этим банком. Специалисты...,15.05.2021 14:50,5.0,благодарность,Отличный банк!\n\t\t\t\t\t\t\t\tЯ давно сотруд...,отличный банк я давно сотрудничаю с этим банко...,отличн банк давн сотруднича эт банк специалист...,отличный банк давно сотрудничать банк специали...
1,https://www.banki.ru/services/responses/bank/r...,БУДЬТЕ ОЧЕНЬ БДИТЕЛЬНЫМИ в этом банке!Не реко...,"Придя в банк,я заняла очередь.Моим вопросом мо...",15.05.2021 14:44,2.0,плохое обслуживание,БУДЬТЕ ОЧЕНЬ БДИТЕЛЬНЫМИ в этом банке!Не реко...,будьте очень бдительными в этом банке не реком...,будьт очен бдительн банк рекоменд прид банк за...,очень бдительный банк рекомендовать прийти бан...
2,https://www.banki.ru/services/responses/bank/r...,"Названивают без повода, не слушают вообще ниче...",Названивают мне с информацией о незнакомом мне...,15.05.2021 13:24,1.0,навязывание,"Названивают без повода, не слушают вообще ниче...",названивают без повода не слушают вообще ничег...,названива повод слуша вообщ хам телефон исполь...,названивать повод слушать вообще ничто хамить ...
3,https://www.banki.ru/services/responses/bank/r...,Благодарность\n\t\t\t\t\t\t\t\t,"Обращался в Банк Тинькофф по поводу акции ""При...",15.05.2021 10:00,5.0,благодарность,Благодарность\n\t\t\t\t\t\t\t\tОбращался в Бан...,благодарность обращался в банк тинькофф по пов...,благодарн обраща банк тинькофф повод акц приве...,благодарность обращаться банк тинькофф повод а...
4,https://www.banki.ru/services/responses/bank/r...,Отвратительное отношение к клиентам\n\t\t\t\t\...,Обратился в банк т.к. появилось желания начать...,14.05.2021 23:23,1.0,плохое обслуживание,Отвратительное отношение к клиентам\n\t\t\t\t\...,отвратительное отношение к клиентам обратился ...,отвратительн отношен клиент обрат банк т появ ...,отвратительный отношение клиент обращаться бан...


Применение результатов лемматизации

Лемматизация.Подготовка выборок

In [11]:
X=df_train['Лемм_текст']
y=df_train['Тематика']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state = 37)

Лемматизация.Naive Bayes

In [12]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])

nb.fit(X_train, y_train)

from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

my_tags = df_train['Тематика'].unique()

from sklearn.metrics import accuracy_score

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred, target_names=my_tags))

accuracy 0.577922077922078
                       precision    recall  f1-score   support

        благодарность       0.57      1.00      0.72       250
  плохое обслуживание       0.00      0.00      0.00        12
          навязывание       0.78      0.16      0.27       112
                 сбой       0.00      0.00      0.00        20
некачественная услуга       0.00      0.00      0.00        44
       отказ в услуге       0.00      0.00      0.00        24

             accuracy                           0.58       462
            macro avg       0.22      0.19      0.16       462
         weighted avg       0.50      0.58      0.46       462



C:\Users\shmel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\shmel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\shmel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Лемматизация. Linear Support Vector Machine

In [13]:
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])

sgd.fit(X_train, y_train)

y_pred = sgd.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

accuracy 0.645021645021645
                       precision    recall  f1-score   support

        благодарность       0.71      0.87      0.78       250
  плохое обслуживание       0.25      0.08      0.12        12
          навязывание       0.57      0.54      0.55       112
                 сбой       0.25      0.05      0.08        20
некачественная услуга       0.53      0.23      0.32        44
       отказ в услуге       0.38      0.33      0.36        24

             accuracy                           0.65       462
            macro avg       0.45      0.35      0.37       462
         weighted avg       0.61      0.65      0.61       462



Лемматизация.Logistic Regression

In [14]:
from sklearn.linear_model import LogisticRegression

logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5)),
               ])

logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

accuracy 0.6493506493506493
                       precision    recall  f1-score   support

        благодарность       0.73      0.86      0.79       250
  плохое обслуживание       0.33      0.17      0.22        12
          навязывание       0.55      0.50      0.52       112
                 сбой       0.33      0.15      0.21        20
некачественная услуга       0.52      0.30      0.38        44
       отказ в услуге       0.42      0.42      0.42        24

             accuracy                           0.65       462
            macro avg       0.48      0.40      0.42       462
         weighted avg       0.62      0.65      0.63       462



C:\Users\shmel\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Применение результатов стемминга

Стемминг. Подготовка выборок

In [15]:
X=df_train['Стем_текст']
y=df_train['Тематика']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state = 37)

Стемминг. Naive Bayes

In [16]:
nb.fit(X_train, y_train)

y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred, target_names=my_tags))

accuracy 0.5670995670995671
                       precision    recall  f1-score   support

        благодарность       0.56      0.99      0.72       250
  плохое обслуживание       0.00      0.00      0.00        12
          навязывание       0.70      0.12      0.21       112
                 сбой       0.00      0.00      0.00        20
некачественная услуга       0.00      0.00      0.00        44
       отказ в услуге       0.00      0.00      0.00        24

             accuracy                           0.57       462
            macro avg       0.21      0.19      0.15       462
         weighted avg       0.47      0.57      0.44       462



C:\Users\shmel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\shmel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\shmel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


 Стемминг. Linear Support Vector Machine

In [17]:
sgd.fit(X_train, y_train)
y_pred = sgd.predict(X_test)
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

accuracy 0.6406926406926406
                       precision    recall  f1-score   support

        благодарность       0.69      0.88      0.77       250
  плохое обслуживание       0.50      0.25      0.33        12
          навязывание       0.54      0.49      0.52       112
                 сбой       0.25      0.05      0.08        20
некачественная услуга       0.62      0.23      0.33        44
       отказ в услуге       0.44      0.33      0.38        24

             accuracy                           0.64       462
            macro avg       0.51      0.37      0.40       462
         weighted avg       0.61      0.64      0.61       462



Стемминг.Logistic Regression

In [18]:
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

accuracy 0.6515151515151515
                       precision    recall  f1-score   support

        благодарность       0.72      0.86      0.78       250
  плохое обслуживание       0.50      0.25      0.33        12
          навязывание       0.55      0.51      0.53       112
                 сбой       0.22      0.10      0.14        20
некачественная услуга       0.62      0.34      0.44        44
       отказ в услуге       0.43      0.38      0.40        24

             accuracy                           0.65       462
            macro avg       0.51      0.41      0.44       462
         weighted avg       0.63      0.65      0.63       462



C:\Users\shmel\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Лучший результат - Стемминг. Логистическая регрессия

Изменение параметров

In [19]:
logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e2, penalty='l2')),#изменили С, теперь результат максимальный
               ])
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

accuracy 0.658008658008658
                       precision    recall  f1-score   support

        благодарность       0.71      0.88      0.79       250
  плохое обслуживание       0.25      0.08      0.12        12
          навязывание       0.57      0.53      0.55       112
                 сбой       0.33      0.10      0.15        20
некачественная услуга       0.63      0.27      0.38        44
       отказ в услуге       0.50      0.38      0.43        24

             accuracy                           0.66       462
            macro avg       0.50      0.37      0.40       462
         weighted avg       0.63      0.66      0.63       462



C:\Users\shmel\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


 Боевое применение

In [20]:
%%time
#парсинг сайта
ssl._create_default_https_context = ssl._create_unverified_context

url = 'https://www.banki.ru/services/responses/bank/sberbank/product/investments/'
r = requests.get(url)

df_sber = pd.DataFrame(columns=['url жалобы', 'Заголовок', 'Текст', 'Время', 'Оценка'])

soup = BeautifulSoup(r.text)

pages = 38
page_url= '?page='
for page in range(1, pages + 2):
    t.sleep(1)
    for article in soup.findAll('article'):
        try:
            href = article.find('a', {'data-test': 'responses-header'}).get('href')
            header = article.find('a', {'data-test': 'responses-header'}).text
            text = article.find('div', {'data-test': 'responses-message'}).text.strip()
            time = article.find('time', {'data-test': 'responses-datetime'}).text.strip()
            
            # optional
            try:
                rating = article.find('span', {'data-test': 'responses-rating-grade'}).text.strip()
            except:
                rating = None
            row = [url[:21]+href[1:], header, text, time, rating]
            df_sber.loc[len(df_sber)] = row
        except:
            pass
    # Взятие следующей страницы
    try:
        if page <= pages:
            r = requests.get(url + page_url + str(page))
            soup = BeautifulSoup(r.text, features='lxml')
    except:
        pass
print(len(df_sber))
df_sber.to_excel('Отзывы_Сбербанк.xlsx')

#Предобработка
df_sber['Заг_и_текст']=df_sber['Заголовок']+df_sber['Текст']
prep_text = [remove_multiple_spaces(remove_numbers(remove_punctuation(text.lower()))) for text in df_sber['Заг_и_текст']]
df_sber['Пред_текст'] = prep_text
df_sber.head()

#Стемминг
stemmed_texts_list = []
for text in df_sber['Пред_текст']:
    tokens = word_tokenize(text)    
    stemmed_tokens = [stemmer.stem(token) for token in tokens if token not in russian_stopwords]
    text = " ".join(stemmed_tokens)
    stemmed_texts_list.append(text)
df_sber['Стем_текст'] = stemmed_texts_list

#Обработка
sber_pred = logreg.predict(df_sber['Стем_текст'])
df_sber['Классификация']=sber_pred
df_sber.drop(['Заг_и_текст', 'Пред_текст', 'Стем_текст'], axis=1, inplace=True)
df_sber.to_excel('sber_broker_clsf.xlsx')

#Сохранение результата
df_sber.to_excel('sber_broker_clsf.xlsx')

975
Wall time: 1min 4s


# Вывод

Для нашей целей наиболее приемлемой моделью является многомерная классификация с использованием метода логистической регрессии.
В то же время точность модели (accuracy) недостаточна.
Причины:
    - малое количество данных (количество обращений по брокерскому обслуживанию на сайте в целом небольшое);
    - в качестве данных использовался заголовок обращения и первые 2-3 предложения, для получение большей информации необходимо проваливаться в каждое сообщение, что занимает больше времени 
    - небольшое количество размеченных данных (требуются дополнительные трудозатраты для классификации каждого обращения).

Пути решения:
    - использовать обращения с других сайтов, например broker-ratings.ru, smartlab.ru и т.д;
    - настроить парсер на "вытягивание" всей сути обращения (быть готовым к увеличению времени работы алгоритма);
    - накопить историю обращений.